In [197]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor

In [228]:
df = pd.read_csv('/kaggle/input/d/camnugent/california-housing-prices/housing.csv')

In [229]:
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [230]:
df.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [231]:
df.duplicated().sum()

np.int64(0)

In [233]:
df = df.drop(['total_bedrooms'], axis=1)

In [234]:
df.sample(5)

,longitude,latitude,housing_median_age,total_rooms,population,households,median_income,median_house_value,ocean_proximity
14137,-117.05,32.74,34.0,2178.0,1193.0,446.0,3.1719,115300.0,NEAR OCEAN
6936,-118.09,34.00,36.0,1722.0,1174.0,335.0,3.0450,160600.0,<1H OCEAN
3867,-118.46,34.17,22.0,6707.0,2620.0,1610.0,3.1478,273700.0,<1H OCEAN
19224,-122.70,38.45,39.0,2015.0,640.0,315.0,4.1734,240500.0,<1H OCEAN
4815,-118.29,34.04,41.0,659.0,1224.0,290.0,2.0817,132500.0,<1H OCEAN


In [235]:
df.dtypes

longitude             float64
latitude              float64
housing_median_age    float64
total_rooms           float64
population            float64
households            float64
median_income         float64
median_house_value    float64
ocean_proximity        object
dtype: object

In [237]:
# ohe = OneHotEncoder(drop='first', sparse_output=False)

ohe_ocean = OneHotEncoder(drop='first', sparse_output=False)
ocean_ohe = ohe_ocean.fit_transform(df[['ocean_proximity']])


ocean_df = pd.DataFrame(
    ocean_ohe,
    columns=ohe_ocean.get_feature_names_out(['ocean_proximity'])
)

df = df.reset_index(drop=True)
ocean_df = ocean_df.reset_index(drop=True)
df = df.drop(['ocean_proximity'], axis=1)
df = pd.concat([df, ocean_df], axis=1)

In [238]:
# std = StandardScaler()

num = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'population', 'households', 'median_income']
# df[num] = std.fit_transform(df[num])

In [239]:
df.sample(5)

,longitude,latitude,housing_median_age,total_rooms,population,households,median_income,median_house_value,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
8441,-118.36,33.91,42.0,1949.0,1184.0,423.0,4.3333,225600.0,0.0,0.0,0.0,0.0
9161,-118.49,34.42,23.0,4166.0,2082.0,743.0,4.4107,213400.0,0.0,0.0,0.0,0.0
17986,-121.96,37.30,20.0,4228.0,2334.0,1007.0,4.3081,227300.0,0.0,0.0,0.0,0.0
11464,-117.99,33.67,17.0,1692.0,903.0,423.0,3.5859,262500.0,0.0,0.0,0.0,0.0
6561,-118.13,34.21,36.0,1449.0,621.0,210.0,6.1824,274100.0,1.0,0.0,0.0,0.0


In [240]:
X = df.drop('median_house_value', axis=1)
y = df['median_house_value']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train[num] = std.fit_transform(X_train[num])
X_test[num] = std.transform(X_test[num])


In [253]:
rf = RandomForestRegressor(
    n_estimators=300,
    max_depth=20,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

print(r2_score(y_test, y_pred))


0.8205406111075174


In [243]:
tree = DecisionTreeRegressor()

tree.fit(X_train, y_train)

y_pred = tree.predict(X_test)

In [244]:
r2_score(y_test, y_pred)

0.6627242665592956

In [245]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

In [246]:
r2_score(y_test, y_pred)

0.6307553884717846

In [226]:
features = [
    'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
    'floors', 'waterfront', 'view', 'condition',
    'sqft_above', 'sqft_basement'
]

df_model = df[features + ['price']]


In [250]:
from xgboost import XGBRegressor

xgb = XGBRegressor(
    
)

In [251]:
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)

r2_score(y_pred, y_test)

0.8071624167560381